### Imports

In [15]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests

### Directory Definitions

In [35]:
data_dir = "html_files/"
formatted_data_dir = "formatted_cases/"
case_data_path = "case_data/cases.csv"
scraped_searches_path = "scraped_searches/"

# Create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Create the directory if it doesn't exist
if not os.path.exists(formatted_data_dir):
    os.makedirs(formatted_data_dir)

### Get case IDs

In [57]:
df = pd.read_csv(case_data_path)
case_IDs = df['What is the file number of the case?'].tolist()
# case_IDs = case_IDs[:2]

In [ ]:
# !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# Set up Selenium WebDriver (you may need to download and configure the appropriate WebDriver for your browser)
driver = webdriver.Chrome()

errors_list = []

for case_ID in case_IDs:
    if os.path.isfile(scraped_searches_path + case_ID + ".html"):
        print(case_ID, "already added, skipping...")
    else:
        try:
            url = "https://www.canlii.org/en/#search/type=decision&id=" + case_ID + "&resultIndex=1"
            driver.get(url)
            # Get all content, including dynamically generated content
            html_content = driver.page_source

            # Save the content to a file
            with open(scraped_searches_path + case_ID + ".html", "w", encoding="utf-8") as file:
                file.write(html_content)

            print(case_ID, "saved.")
        except:
            errors_list.append(case_ID)

# Close the WebDriver
driver.quit()


In [60]:
urls_df = pd.DataFrame(columns=['case_ID', 'case_URL'])

for file in os.listdir(scraped_searches_path):
    if os.path.isfile(scraped_searches_path + file):
        with open(scraped_searches_path + file) as f:
            html = f.read()
            soup = BeautifulSoup(html, "html.parser")
            # Find the <a> element within the <span> element
            a_element = soup.find('span', class_='name').find('a')

            # Extract the value of the href attribute
            href = a_element['href']
            case_url = 'https://www.canlii.org/' + href

            data = {'case_ID': [os.path.splitext(file)[0]], 'case_URL': [url]}
            urls_df = urls_df.append(pd.DataFrame(data), ignore_index=True)

In [61]:
urls_df

case_ID                                           case_URL
0    TSL-95864-18  https://www.canlii.org/en/#search/type=decisio...
1    TEL-85031-17  https://www.canlii.org/en/#search/type=decisio...
2    TEL-86233-17  https://www.canlii.org/en/#search/type=decisio...
3    TSL-95874-18  https://www.canlii.org/en/#search/type=decisio...
4    CEL-68020-17  https://www.canlii.org/en/#search/type=decisio...
..            ...                                                ...
675  CEL-72966-18  https://www.canlii.org/en/#search/type=decisio...
676  TEL-87479-18  https://www.canlii.org/en/#search/type=decisio...
677  TEL-94512-18  https://www.canlii.org/en/#search/type=decisio...
678  TNL-21505-19  https://www.canlii.org/en/#search/type=decisio...
679  TSL-94942-18  https://www.canlii.org/en/#search/type=decisio...

[680 rows x 2 columns]

In [ ]:
### Get all HTML files from 

### Scraping from folder

In [55]:
for file in os.listdir(data_dir):
    if os.path.isfile(data_dir + file):
        if os.path.isfile(formatted_data_dir + os.path.splitext(file)[0] + ".txt"):
            print(file, "already added, skipping...")
        else:
            print("Adding ", file, "...")
            with open(data_dir + file) as f:
                html = f.read()
            soup = BeautifulSoup(html, "html.parser")

            # find metadata
            document_meta = soup.find("div", {"id": "documentMeta"}) 
            meta_items = document_meta.find_all("div", {"class": "row py-1"})
            meta_data = []
            for meta_item in meta_items:
                meta_data.append('\t'.join([x.text for x in meta_item.findChildren()]))
            # print(meta_data)
            # find text
            document_body = soup.find("div", {"class": "documentcontent"}).get_text()

            # write to file
            with open(formatted_data_dir + os.path.splitext(file)[0] + '.txt', 'w') as file:
                file.write('Metadata:\n')
                file.write('\n'.join(meta_data))
                file.write('Content:\n')
                file.write(document_body)

Adding  CEL-87788-19.html ...
TNL-22838-19.html already added, skipping...
